In [2]:
import os
import re
import io
import requests
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Recurrent Neural Networks
* Recurrent networks share their weights over time, thus "recurrent"
* This allows them to have a kind of "memory" about what happened earlier in the sequence.

<img src="../pics/RNN_loop_unrolled.jpg">

* Here we create an RNN composed of a layer of five recurrent neurons using ReLU for our activation function. 
* In this simple case, we are assuming that the RNN runs over only two-time steps, taking input vectors of size 3 at each time step. 

In [3]:
n_inputs = 3
n_neurons = 5

X1 = tf.placeholder(tf.float32, [None, n_inputs])
X2 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.get_variable("Wx", shape=[n_inputs,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

Wy = tf.get_variable("Wy", shape=[n_neurons,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

b = tf.get_variable("b", shape=[1,n_neurons], dtype=tf.float32, initializer=None, regularizer=None, trainable=True, collections=None)

Y1 = tf.nn.relu(tf.matmul(X1, Wx) + b)
Y2 = tf.nn.relu(tf.matmul(Y1, Wy) + tf.matmul(X2, Wx) + b)

init_op = tf.global_variables_initializer()

* This network looks much like a two-layer feedforward neural network, but both layers share the same weights and bias vectors. 
* Also, we feed inputs at each layer and receive outputs from each layer.

In [4]:
# Mini-batch: instance 0,instance 1,instance 2,instance 3
X1_batch = np.array([[0, 2, 3], [2, 8, 9], [5, 3, 8], [3, 2, 9]]) # t = 0
X2_batch = np.array([[5, 6, 8], [1, 0, 0], [8, 2, 0], [2, 3, 6]]) # t = 1

* These mini-batches contain four instances, each with an input sequence composed of exactly two inputs. 
* At the end, Y1_val and Y2_val contain the outputs of the network at both time steps for all neurons and all instances in the mini-batch.

In [5]:
with tf.Session() as sess:
	init_op.run()
	Y1_val, Y2_val = sess.run([Y1, Y2], feed_dict={X1: X1_batch, X2: X2_batch})

print(Y1_val) # output at t = 0
print()
print(Y2_val) # output at t = 1

[[ 3.4328816   2.6566072   0.27672583  0.          0.        ]
 [10.837278   10.352432    1.8948631   1.8187337   0.        ]
 [10.265583    8.670134    2.654381    2.2625995   0.        ]
 [ 9.558879    8.379587    0.49371815  1.1972008   0.        ]]

[[13.087704  11.129164   2.7504628  1.9635859  0.       ]
 [ 7.688302   3.9405813  0.         0.         0.       ]
 [10.975104   7.2570753  4.2138577  3.329887   0.7585387]
 [12.749466  10.189095   0.         0.         0.       ]]


### Using contrib

In [6]:
tf.reset_default_graph()
n_inputs = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# note that here instance one is not as long as the other sequences
# so we padded it with a zero vector
X_batch = np.array([
                   [[0, 2, 3], [2, 8, 9]], # instance 0
                   [[5, 6, 8], [0, 0, 0]], # instance 1 (padded with a zero vector)
                   [[6, 7, 8], [6, 5, 4]], # instance 2
                   [[8, 2, 0], [2, 3, 6]], # instance 3
                  ])
seq_length_batch = np.array([3, 4, 3, 5])
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
        init_op.run()
        outputs_val, states_val = sess.run([output_seqs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

print(outputs_val)

[[[-0.6242257   0.54043114  0.98715794 -0.9825246   0.9693041 ]
  [-0.9560957   0.9865212   0.99997306 -0.9999989   0.9999999 ]]

 [[-0.67390215  0.9120273   0.99933183 -0.9999703   1.        ]
  [ 0.45531037  0.33025947 -0.95130485  0.41533625 -0.05823964]]

 [[-0.6646165   0.9239195   0.99911267 -0.9999819   1.        ]
  [ 0.47305995  0.8370427  -0.7788391  -0.98684967  0.9999991 ]]

 [[ 0.9198561  -0.01989019 -0.99921227  0.64939207  0.99997777]
  [-0.23007126 -0.20524526  0.9995201  -0.9977988   0.9999862 ]]]
